# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [16]:
# import libraries
import re

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords', 'vader_lexicon'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import joblib
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.preprocessing import Normalizer
from sklearn.kernel_approximation import Nystroem
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package punkt to C:\Users\Disha M
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Disha M
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Disha M
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Disha M
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [17]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql('etl', engine)
X = df.message.values
Y = df.iloc[:,4:].copy()

### 2. Write a tokenization function to process your text data

In [18]:
def tokenize(text):
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    words = word_tokenize(text.lower())
    words = [w for w in words if w not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    for w in words:
        lemmatizer.lemmatize(w)
        lemmatizer.lemmatize(w, pos='v')
        lemmatizer.lemmatize(w, pos='a')
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [19]:
!pip install vadersentiment
!pip3 install vadersentiment
pipeline = Pipeline([
    ('nlp', FeatureUnion([
        ('importance', Pipeline([
            ('bow', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
         ('sentiment', vadersentiment())
        
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])

NameError: name 'vadersentiment' is not defined

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
pipeline.fit(X_train, Y_train)

NameError: name 'pipeline' is not defined

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
Y_pred = pipeline.predict(X_test)
print(classification_report(Y_test, Y_pred))

#('kernel_approximation', Nystroem(random_state = 42)),
#'kernel_approximation__n_components': [100, 200],

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline = Pipeline([
    ('nlp', FeatureUnion([
        ('importance', Pipeline([
            ('bow', CountVectorizer(tokenizer=tokenize, min_df=5, ngram_range=(1,2))),
            ('tfidf', TfidfTransformer())
        ])),
         ('sentiment', VaderSentiment(tokenize))
    ])),
    ('scale', Normalizer()),
    ('clf', MultiOutputClassifier(LinearSVC(random_state=42, class_weight='balanced')))
])


parameters = {
    'nlp__importance__tfidf__sublinear_tf': [True, False]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=2)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
cv.fit(X_train, Y_train)

In [ ]:
pd.DataFrame(cv.cv_results_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
Y_pred = cv.predict(X_test)
print(classification_report(Y_test, Y_pred))

In [ ]:
pipeline = Pipeline([
    ('nlp', FeatureUnion([
        ('importance', Pipeline([
            ('bow', CountVectorizer(tokenizer=tokenize, min_df=5, ngram_range=(1,2))),
            ('tfidf', TfidfTransformer())
        ])),
         ('sentiment', VaderSentiment(tokenize))
    ])),
    ('scale', Normalizer()),
    ('kernel', Nystroem(random_state=42)),
    ('clf', MultiOutputClassifier(LinearSVC(random_state=42, class_weight='balanced')))
])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
pipeline.fit(X_train, Y_train)

In [ ]:
Y_pred = pipeline.predict(X_test)
print(classification_report(Y_test, Y_pred))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

class VaderSentiment(BaseEstimator, TransformerMixin):
    ''' sentiment analysis using VADER method from: 
        Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for
            Sentiment Analysis of Social Media Text. Eighth International Conference on
            Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.
    '''
    
    def __init__(self, tokenizer=None):
        self.vader = SentimentIntensityAnalyzer()
        self.tokenizer = tokenizer

    def tokenize(self, text):
        text = re.sub('[^a-zA-Z0-9]', ' ', text)
        words = word_tokenize(text.lower())
        words = [w for w in words if w not in stopwords.words('english')]
        lemmatizer = WordNetLemmatizer()
        for w in words:
            lemmatizer.lemmatize(w)
            lemmatizer.lemmatize(w, pos='v')
            lemmatizer.lemmatize(w, pos='a')
        return words

    def analyze(self, text):
        if self.tokenizer is None:
            text = ' '.join(self.tokenize(text))
        else:
            text = ' '.join(self.tokenizer(text))
        scores = self.vader.polarity_scores(text)
        return pd.Series([scores['neg'], scores['neu'], scores['pos']])

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        scores = pd.Series(X).apply(self.analyze)
        scores.columns = ['neg', 'neu', 'pos']
        return scores

### 9. Export your model as a pickle file

In [ ]:
joblib.dump(pipeline, 'model.pkl', compress=1)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.